<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [1]:
# Para gestionar ficheros
# -----------------------------------------------------------------------
import os
import glob

# Para trabajar con access
# -----------------------------------------------------------------------
import subprocess


# Para trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd


# Para trabajar con regex
# -----------------------------------------------------------------------
import re

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
ruta_origen = r"C:\Users\Administrador\Desktop\RCM\laboratorio-modulo5-leccion01-elt-extraccion\data"

### Cargamos los archivos de demanda

In [ ]:
ruta_origen = r"C:\Users\Administrador\Desktop\RCM\laboratorio-modulo5-leccion01-elt-extraccion\data"

In [5]:
archivos = os.listdir(ruta_origen)
archivos

['2019_demanda_evolucion_Andalucía_4.csv',
 '2019_demanda_evolucion_Aragón_5.csv',
 '2019_demanda_evolucion_Canarias_8742.csv',
 '2019_demanda_evolucion_Cantabria_6.csv',
 '2019_demanda_evolucion_Castilla - La Mancha_7.csv',
 '2019_demanda_evolucion_Castilla y León_8.csv',
 '2019_demanda_evolucion_Cataluña_9.csv',
 '2019_demanda_evolucion_Ceuta_8744.csv',
 '2019_demanda_evolucion_Comunidad de Madrid_13.csv',
 '2019_demanda_evolucion_Comunidad Foral de Navarra_14.csv',
 '2019_demanda_evolucion_Comunitat Valenciana_15.csv',
 '2019_demanda_evolucion_Extremadura_16.csv',
 '2019_demanda_evolucion_Galicia_17.csv',
 '2019_demanda_evolucion_Illes Balears_8743.csv',
 '2019_demanda_evolucion_La Rioja_20.csv',
 '2019_demanda_evolucion_Melilla_8745.csv',
 '2019_demanda_evolucion_País Vasco_10.csv',
 '2019_demanda_evolucion_Principado de Asturias_11.csv',
 '2019_demanda_evolucion_Región de Murcia_21.csv',
 '2019_generacion_estructura_Andalucía_4.csv',
 '2019_generacion_estructura_Aragón_5.csv',
 '2

In [8]:
# Diccionario para almacenar los DataFrames de demanda
diccionario_dataframes_demanda = {}

for archivo in tqdm(archivos):
    # seleccionamos solo los archivos de demanda
    if "demanda" in archivo.lower():
        # Extraemos con regex el nombre de la comunidad y el número final del archivo
        match = re.search(r"demanda_evolucion_(.*)_(\d+)\.csv", archivo)
        if match:
            comunidad = match.group(1).replace(" ", "_")  # Remplazamos los espacios con guiones bajos
            numero = match.group(2)
            nombre_df = f"df_{comunidad}_{numero}"
            # definimos la ruta para abrir cada uno de los ficheros y realizar las transformaciones
            ruta_fichero_demanda = os.path.join(ruta_origen, archivo)
            try:
                df = pd.read_csv(ruta_fichero_demanda)
                diccionario_dataframes_demanda[nombre_df] = df  # Guardamos en el diccionario con el nombre específico
                print(f"Cargado {nombre_df} correctamente.")
            except Exception as e:
                print(f"Error al cargar {archivo}: {e}")

100%|██████████| 112/112 [00:00<00:00, 661.63it/s]

Cargado df_Andalucía_4 correctamente.
Cargado df_Aragón_5 correctamente.
Cargado df_Canarias_8742 correctamente.
Cargado df_Cantabria_6 correctamente.
Cargado df_Castilla_-_La_Mancha_7 correctamente.
Cargado df_Castilla_y_León_8 correctamente.
Cargado df_Cataluña_9 correctamente.
Cargado df_Ceuta_8744 correctamente.
Cargado df_Comunidad_de_Madrid_13 correctamente.
Cargado df_Comunidad_Foral_de_Navarra_14 correctamente.
Cargado df_Comunitat_Valenciana_15 correctamente.
Cargado df_Extremadura_16 correctamente.
Cargado df_Galicia_17 correctamente.
Cargado df_Illes_Balears_8743 correctamente.
Cargado df_La_Rioja_20 correctamente.
Cargado df_Melilla_8745 correctamente.
Cargado df_País_Vasco_10 correctamente.
Cargado df_Principado_de_Asturias_11 correctamente.
Cargado df_Región_de_Murcia_21 correctamente.
Cargado df_Andalucía_4 correctamente.
Cargado df_Aragón_5 correctamente.
Cargado df_Canarias_8742 correctamente.
Cargado df_Cantabria_6 correctamente.
Cargado df_Castilla_-_La_Mancha_7 corr

### Cargamos los archivos de generacion electrica

In [ ]:
# Diccionario para almacenar los DataFrames de generacion
diccionario_dataframes_generacion = {}

for archivo in tqdm(archivos):
    # seleccionamos solo los archivos de generacion
    if "generacion" in archivo.lower():
        # Extraemos con regex el nombre de la comunidad y el número final del archivo
        match = re.search(r"generacion_estructura_(.*)_(\d+)\.csv", archivo)
        if match:
            comunidad = match.group(1).replace(" ", "_")  # Remplazamos los espacios con guiones bajos
            numero = match.group(2)
            nombre_df = f"df_{comunidad}_{numero}"
            # definimos la ruta para abrir cada uno de los ficheros y realizar las transformaciones
            ruta_fichero_generacion = os.path.join(ruta_origen, archivo)
            try:
                df = pd.read_csv(ruta_fichero_generacion)
                diccionario_dataframes_generacion[nombre_df] = df  # Guardamos en el diccionario con el nombre específico
                print(f"Cargado {nombre_df} correctamente.")
            except Exception as e:
                print(f"Error al cargar {archivo}: {e}")

### Cargamos los archivos demográficos del INE

In [14]:
ruta_origen = r"C:\Users\Administrador\Desktop\RCM\laboratorio-modulo5-leccion01-elt-extraccion\data\ine"

In [15]:
archivos = os.listdir(ruta_origen)

In [16]:
archivos

['03001 (1).csv',
 '03001 (2).csv',
 '03001.csv',
 '67284 (1).csv',
 '67284 (2).csv',
 '67284.csv']

In [ ]:
diccionario_dataframes_demograficos = {}

for archivo in tqdm(archivos):
    # seleccionamos solo los archivos de generacion
    if "03001" in archivo():
        
        if match:
            comunidad = match.group(1).replace(" ", "_")  # Remplazamos los espacios con guiones bajos
            numero = match.group(2)
            nombre_df = f"df_{comunidad}_{numero}"
            # definimos la ruta para abrir cada uno de los ficheros y realizar las transformaciones
            ruta_fichero_generacion = os.path.join(ruta_origen, archivo)
            try:
                df = pd.read_csv(ruta_fichero_generacion)
                diccionario_dataframes_generacion[nombre_df] = df  # Guardamos en el diccionario con el nombre específico
                print(f"Cargado {nombre_df} correctamente.")
            except Exception as e:
                print(f"Error al cargar {archivo}: {e}")